# Gerekli Kütüphaneler

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque
from sklearn.datasets import make_blobs
from sklearn import metrics

seed = 42
np.random.seed(seed)
random.seed(seed)

# Veri Seti 

In [2]:
X, y = make_blobs(n_samples=100, centers=3, n_features=2, random_state=0)

# Affinity Propagation

In [3]:
class AffinityPropagation:
    def __init__(self, damping=0.9, max_iter=100, convergence_iter=15):
        # Damping faktoru (0.5 < damping < 1.0)
        self.damping = damping
        # Maksimum iterasyon sayisi
        self.max_iter = max_iter
        # Konsensus iterasyon sayisi
        self.convergence_iter = convergence_iter

    def fit(self, X):
        n_samples = X.shape[0]
        # Baslangicta tercih negativ en kucuk degerlerden olusur
        preference = np.median(X)

        # Sorumluluk (responsibility) matrisi
        R = np.zeros((n_samples, n_samples))
        # Kullanilabilirlik (availability) matrisi
        A = np.zeros((n_samples, n_samples))

        # Benzerlikler guncellenir
        S = X.copy()
        np.fill_diagonal(S, preference)

        for iteration in range(self.max_iter):
            # Sorumluluk guncellemesi
            for i in range(n_samples):
                for k in range(n_samples):
                    max_s = np.max(A[i, :] + S[i, :])
                    R[i, k] = self.damping * R[i, k] + (1 - self.damping) * (S[i, k] - max_s)

            # Kullanilabilirlik guncellemesi
            for i in range(n_samples):
                for k in range(n_samples):
                    if i == k:
                        A[i, k] = self.damping * A[i, k] + (1 - self.damping) * np.sum(np.maximum(R[:, k], 0))
                    else:
                        A[i, k] = self.damping * A[i, k] + (1 - self.damping) * np.minimum(0, R[k, k] + np.sum(np.maximum(R[:, k], 0) - R[i, k]))

            # Konsensus kontrolu
            exemplars = np.where((A + R).diagonal() > 0)[0]

            if len(exemplars) > 0 and iteration >= self.convergence_iter:
                break

        # Kume merkezlerini (exemplar) belirlenir
        cluster_centers_indices = np.where((A + R).diagonal() > 0)[0]

        # Kumeleri atanir
        labels = np.zeros(n_samples)
        for i in range(n_samples):
            labels[i] = np.argmax(S[i, cluster_centers_indices])

        self.cluster_centers_indices_ = cluster_centers_indices
        self.labels_ = labels

    def predict(self, X):
        return self.labels_

# Eğitim

In [4]:
similarity_matrix = -metrics.pairwise_distances(X)

In [5]:
ap = AffinityPropagation(damping=0.5, max_iter=200)
ap.fit(similarity_matrix)

# Sonuçlar

In [6]:
cluster_centers = ap.cluster_centers_indices_

In [11]:
labels = ap.labels_

In [12]:
print(f"Homogeneity: {metrics.homogeneity_score(y, labels):.3f}")
print(f"Completeness: {metrics.completeness_score(y, labels):.3f}")
print(f"V-measure: {metrics.v_measure_score(y, labels):.3f}")
print(f"Adjusted Rand Index: {metrics.adjusted_rand_score(y, labels):.3f}")
print(f"Adjusted Mutual Information: {metrics.adjusted_mutual_info_score(y, labels):.3f}")
print(f"Silhouette Coefficient: {metrics.silhouette_score(X, labels):.3f}")

Homogeneity: 0.632
Completeness: 0.378
V-measure: 0.473
Adjusted Rand Index: 0.296
Adjusted Mutual Information: 0.437
Silhouette Coefficient: 0.121
